In [3]:
import numpy as np
import random


In [4]:

#### Miscellaneous functions
def sigmoid(z):
    """The sigmoid function."""
    return 1.0 / (1.0 + np.exp(-z))


def sigmoid_prime(z):
    """Derivative of the sigmoid function."""
    return sigmoid(z) * (1 - sigmoid(z))


In [5]:
class Network(object):

    def __init__(self, sizes):
        # Initialize a neural network with given layer sizes
        
        self.num_layers = len(sizes)   # Number of layers in the network
        self.sizes = sizes             # List of layer sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        # Initialize biases for all layers except the input layer
        self.weights = [np.random.randn(y, x)
                        for x, y in zip(sizes[:-1], sizes[1:])]
        # Initialize weights for connections between layers


    def feedforward(self, a):
        # Perform feedforward propagation and return the output
        
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a) + b)  # Apply weights, biases, and activation function
        return a


    def SGD(self, training_data, epochs, mini_batch_size, eta,
            test_data=None):
        # Stochastic Gradient Descent for training the network
        
        if test_data:
            n_test = len(test_data)  # Number of test examples
        n = len(training_data)       # Number of training examples

        for j in range(epochs):  # Loop over epochs
            random.shuffle(training_data)  # Shuffle training data
            mini_batches = [
                training_data[k : k + mini_batch_size]
                for k in range(0, n, mini_batch_size)]
            # Create mini-batches for SGD
            
            for batch in mini_batches:  # Loop over mini-batches
                self.update_mini_batch(batch, eta)  # Update weights and biases using backpropagation
                
            if test_data:
                print("Epoch {0}: {1} / {2}".format(
                    j, self.evaluate(test_data), n_test))
            else:
                print("Epoch {0} complete".format(j))


    def update_mini_batch(self, mini_batch, eta):
        # Update the network's weights and biases using gradient descent
        
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        # Initialize gradient arrays
        
        for x, y in mini_batch:  # Loop over mini-batch examples
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            # Calculate gradients using backpropagation
            nabla_b = [nb + dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw + dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        
        self.weights = [w - (eta / len(mini_batch)) * nw
                        for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b - (eta / len(mini_batch)) * nb
                       for b, nb in zip(self.biases, nabla_b)]
        # Update weights and biases using calculated gradients


    def backprop(self, x, y):
        # Calculate gradients for weights and biases using backpropagation
        
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        # Initialize gradient arrays
        
        # Feedforward
        activation = x
        activations = [x]  # Store layer activations
        zs = []  # Store z vectors
        
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation) + b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
        # Perform feedforward propagation to store activations and z vectors
        
        # Backward pass
        delta = self.cost_derivative(activations[-1], y) * sigmoid_prime(zs[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        # Calculate gradients for output layer
        
        for l in range(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l + 1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l - 1].transpose())
        # Loop over layers in reverse order, calculate gradients for each layer
        
        return (nabla_b, nabla_w)


    def evaluate(self, test_data):
        # Evaluate the network's performance on test data
        
        test_results = [(np.argmax(self.feedforward(x)), y)
                        for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)


    def cost_derivative(self, output_activations, y):
        # Calculate the derivative of the cost function with respect to output activations
        
        return (output_activations - y)


In [6]:
net = Network([2, 3, 1])

In [11]:
net.weights

[array([[ 1.62120468,  0.29905807],
        [ 0.50710409,  0.72349781],
        [-0.1232402 ,  1.27555803]]),
 array([[-1.07752149,  0.9412284 ,  0.95035008]])]

In [13]:
import mnist_loader

training_data, validation_data, test_data = mnist_loader.load_data_wrapper()

In [14]:
net =Network([784, 30, 10])
net.SGD(training_data, 30, 10, 3.0, test_data=test_data)

Epoch 0: 8246 / 10000
Epoch 1: 9238 / 10000
Epoch 2: 9254 / 10000
Epoch 3: 9269 / 10000
Epoch 4: 9332 / 10000
Epoch 5: 9360 / 10000
Epoch 6: 9349 / 10000
Epoch 7: 9397 / 10000
Epoch 8: 9441 / 10000
Epoch 9: 9453 / 10000
Epoch 10: 9409 / 10000
Epoch 11: 9408 / 10000
Epoch 12: 9452 / 10000
Epoch 13: 9456 / 10000
Epoch 14: 9452 / 10000
Epoch 15: 9467 / 10000
Epoch 16: 9465 / 10000
Epoch 17: 9472 / 10000
Epoch 18: 9466 / 10000
Epoch 19: 9463 / 10000
Epoch 20: 9472 / 10000
Epoch 21: 9489 / 10000
Epoch 22: 9446 / 10000
Epoch 23: 9486 / 10000
Epoch 24: 9483 / 10000
Epoch 25: 9494 / 10000
Epoch 26: 9487 / 10000
Epoch 27: 9494 / 10000
Epoch 28: 9496 / 10000
Epoch 29: 9481 / 10000
